# 2.1 项目组织

之前我们的项目基本没用到什么组织,无非不过是一个文件专注内容一个文件启动外带一些静态文件和模板而已.这样的组织结构已经可用,对于多数微型app来说也不会有啥不便,不过精益求精,我们来规范下项目目录

这章节我们要做一个留言板,我们会根据实际遇到的需求一版一版的晚上这个app,最终让它确实可用.

首先我们以之前的helloworld项目作为模板将其中的东西都拷贝进新建的项目msgboard中.
并将所有文件都规范化,引入`__future__`模块,固定编码为utf-8,写下作者,日期和授权协议,并做好文档工作

## 配置文件和结构


我们的项目有时候会根据不同的环境改变配置,比如调试用的数据库和真正运行就不一样所以我们可以把配置数据放入一个专用的文件中,然后作为全局变量引入app来做设置

In [44]:
%%writefile ../codes/msgboard/config.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals

import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config:
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'hard to guess string' 
    
    @staticmethod
    def init_app(app):
        pass
    

class DevelopmentConfig(Config): 
    
    DEBUG = True
         
class TestingConfig(Config): 
    TESTING = True
    
class ProductionConfig(Config):
    pass

config = {
    'development': DevelopmentConfig,
    'testing': TestingConfig, 
    'production': ProductionConfig,
    'default': DevelopmentConfig
}

    
    

Overwriting ../codes/msgboard/config.py


接着我们把之前的静态文件,模板都拷贝过去,并把文件的模板写好随时可以开始

In [60]:
%%writefile ../codes/msgboard/manager.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A startup manager of the application.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

应用的启动文件

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""

__author__ = "Huang Sizhe"
__date__ = "22/01/2016"

import os
import sys
from flask.ext.script import Manager

root = os.path.dirname(__file__)

from app import app

manager = Manager(app)

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/msgboard/manager.py


In [75]:
%%writefile ../codes/msgboard/app.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A message board appliation.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

留言板应用

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""
#=================导入模块=================
from flask import Flask,render_template,make_response

from flask.ext.bootstrap import Bootstrap

from flask_wtf.csrf import CsrfProtect
from flask.ext.wtf import Form
from wtforms import StringField, SubmitField 
from wtforms.validators import Required

#=================载入插件=================
bootstrap = Bootstrap()
csrf = CsrfProtect()

#=================应用设置=================
from config import config
def create_app(config_name):
    app = Flask(__name__)
    app.config.from_object(config[config_name])
    config[config_name].init_app(app)
    bootstrap.init_app(app)
    csrf.init_app(app)
    return app

import os
app = create_app(os.getenv('FLASK_CONFIG') or 'default')    

#================主体=====================


##----------------表单--------------------

"""class aForm(Form):
    string = StringField('How much RMB to change?', validators=[Required()])
    submit = SubmitField('Submit')"""
    
##----------------模板过滤器---------------

"""@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    pass"""

##----------------通用行为-----------------

"""@app.before_request
def before_request():
    pass
"""
##----------------错误页面-----------------
"""@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}"""
##----------------路由--------------------

@app.route('/',methods = ["GET","POST"])
def hello():
    """aform = aForm()
    
    if aform.validate_on_submit():
        pass"""
    response = make_response(render_template('app/index.html'))
    return response



Overwriting ../codes/msgboard/app.py


In [73]:
%%writefile ../codes/msgboard/templates/app/index.html
{% extends "/base.html" %}
{% block title %}MSG BOARD{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <h1>MSG BOARD!</h1>
        </div>      
    </div>    
    
{{ super() }}
{% endblock %}

Overwriting ../codes/msgboard/templates/app/index.html


In [74]:
!python3 ../codes/msgboard/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [22/Jan/2016 16:21:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2016 16:21:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jan/2016 16:21:03] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
127.0.0.1 - - [22/Jan/2016 16:21:03] "GET /apple-touch-icon.png HTTP/1.1" 404 -
^C


## 使用虚拟环境

为啥要使用虚拟环境呢?因为这样的话便于固定使用环境,移植会方便许多.我们的项目最终目的都是希望放到服务器上为大众提供服务的,因此可移植性是非常重要的一个方面.打开`terminal`,使用pyvenv创建一个调试用的虚拟环境

    cd your_path
    pyvenv env
  
激活你的运行环境:

    source env/bin/activate
    
接着为你的虚拟环境安装必要地包.可以参考之前微项目部分每章最后的总结

目前用到的包有:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单


## 利用git做版本控制

最简单的方法是使用"github Desktop"来做.毕竟有gui,选项又少,比较方便.具体的可以看各种教程,我之前也有写过一篇,可以作为参考

## 确定使用的开源协议

一般来说我们都用开源协议来规范后续开发者的行为(万一火了呢手动斜眼),常见的许可证有那么几种,我之前有文章提到过如何选择,总之找个放到文件根目录!
我选择MIT许可证

In [21]:
%%writefile ../codes/msgboard/License

Copyright (C) 22/01/2016 Huang Sizhe

　　Permission is hereby granted, free of charge,to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
　　
　　The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Writing ../codes/msgboard/License


##  第三方包使用需求文档(requirement.txt)

我们最好时常用pip工具固定下使用包的版本,这样才好更好的移植

使用

    $ pip freeze > requirements.txt 
    
    
来固定使用的包,下次移植的时候只要在新的环境中使用

    $ pip install -r requirements.txt
    
就可以原样
不变的安装需要的依赖了
    

## 测试文件

为了找bug我们还应该有一个测试文件专门用来做测试(目前先不管这块,后面会一起讨论)

In [22]:
%%writefile ../codes/msgboard/test.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals

import nose
from app import app


Overwriting ../codes/msgboard/test.py


## 基本的组织形式:

    msgboard\
            |--app.py
            |--config.py
            |--manager.py
            |--test.py
            |--License
            |--requirement.txt
            |--env\
            |--static\
            |--templates\
                        |--base.html
                        |--blocks\
                        |        |--footer.html
                        |        |--mainnavbar.html
                        |
                        |--index.html